In [1]:
import torchaudio
import torch

In [2]:
from soundstream import StreamableModel

model = StreamableModel(
        batch_size=32,
        sample_rate=16_000,
        segment_length=32270,
        padding='same',
        dataset='librispeech')

ckpt = '/ari/users/ibaskaya/projeler/sstream/lightning_logs/version_10/checkpoints/last.ckpt'
infile = '/ari/users/ibaskaya/projeler/sstream/data/testdata/908_157963_908-157963-0028.in.wav'
outfile = '/ari/users/ibaskaya/projeler/sstream/data/testdata/outputs/cikti.wav'
model.load_state_dict(torch.load(ckpt,'cpu')['state_dict'])
_ = model.eval()

x, sr = torchaudio.load(infile)
x, sr = torchaudio.functional.resample(x, sr, 16000), 16000
with torch.no_grad():
    y = model.debugencode(x.unsqueeze(0))
    # y = y[:, :, :4]  # if you want to reduce code size.
    z = model.decode(y)
#torchaudio.save(outfile, z.squeeze(0), sr)

/ari/users/ibaskaya/projeler/sstream/soundstream.py:236: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
/ari/users/ibaskaya/projeler/sstream/soundstream.py:289: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
/ari/users/ibaskaya/.local/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/tmp/ipykernel_2860476/1136472220.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.c

AttributeError: 'tuple' object has no attribute 'shape'

In [3]:
inn,x,y,z,codes,loss = y

In [4]:
inn.shape,x.shape,y.shape,z.shape,codes.shape,loss.shape

(torch.Size([1, 1, 79280]),
 torch.Size([1, 512, 248]),
 torch.Size([1, 248, 512]),
 torch.Size([1, 248, 512]),
 torch.Size([1, 248, 8]),
 torch.Size([]))

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomSoftmaxEmbedding(nn.Module):
    def __init__(self, num_embeddings=512, embedding_dim=512, 
                 start_t=10.0, end_t=0.0, total_epochs=100):
        super().__init__()
        self.embeddings = nn.Parameter(torch.randn(num_embeddings, embedding_dim))
        
        # annealing parameters
        self.start_t = start_t
        self.end_t = end_t
        self.total_epochs = total_epochs
        self.current_epoch = 0

    def update_epoch(self, epoch: int):
        """Call at start of each epoch"""
        self.current_epoch = min(epoch, self.total_epochs)

    def forward(self, logits: torch.Tensor):
        """
        logits: (B, num_embeddings, seq_len)
        output: (B, embedding_dim, seq_len)
        """
        # decay t linearly
        t = self.start_t - (self.start_t - self.end_t) * (self.current_epoch / self.total_epochs)

        # apply custom activation
        softened_logits = t + F.relu6(logits)                    # (B, N, L)
        attn_weights = F.softmax(softened_logits, dim=1)         # (B, N, L)

        # Weighted sum over embeddings
        # attn_weights: (B, N, L)
        # embeddings: (N, M)
        # -> einsum: bnl,nm -> bml
        output = torch.einsum("bnl,nm->bml", attn_weights, self.embeddings)

        return output


In [7]:
batch_size = 1
num_embeddings = 512
embedding_dim = 512
seq_len = 248

layer = CustomSoftmaxEmbedding(num_embeddings, embedding_dim, total_epochs=50)

logits = torch.randn(batch_size, num_embeddings, seq_len)

layer.update_epoch(0)  # epoch 0
out = layer(logits)
print(out.shape)  # (1, 512, 248)

torch.Size([1, 512, 248])
